In [1]:
import requests, os, zipfile
import glob
import pickle as pk
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix

# Path to the events archive
dataset_zip = '/tmp/events.zip'

# Path to the extracted events
dataset_dir = '/tmp/events/'

# Path to the dataset file
dataset_pkl = '/tmp/dataset.zip'

In [2]:
#@title Dataset Download

import requests, os, zipfile

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)
    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)
    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value
    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768
    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

# Download the dataset
download_file_from_google_drive('1ZPgbhToHLXfLg2IPl0QkxgP6I1TI-KdU', dataset_zip)

# Unzip the dataset
with zipfile.ZipFile(dataset_zip, 'r') as zip_ref:
    zip_ref.extractall(dataset_dir)

In [3]:
# Lists for the sequences
X, y, w = [], [], []

# Particle types to labels
particle_types = {11: 1, 13: 2, 211: 3, 321: 4, 2212: 5}

# Class weights
# class_weight = {
#     11: 74.98247291268324,
#     13: 190.21422797089733,
#     211: 0.2596940335324768,
#     321: 1.524691718019997,
#     2212: 2.1059249977624632
#      }

class_weight = {
    11: 1,
    13: 1,
    211: 1,
    321: 1,
    2212: 1
     }

# Paths to the event files
pkl_paths = glob.glob(dataset_dir + '*.pkl')

# Append the sequences to the lists
for pkl_path in pkl_paths:
  # Unpickle the event file
  pkl_file = open(pkl_path, 'rb')
  event = pk.load(pkl_file)
  pkl_file.close()

  # Extract and preprocess the sequences
  image = np.stack(event[0]) / 13.
  label = np.stack(list(map(lambda x: particle_types[x], event[1])))
  weight = np.stack(list(map(lambda x: class_weight[x], event[1])))

  # Append the sequences
  X += [image]
  y += [label]
  w += [weight]

In [4]:
# Find the maximum sequence length
max_seq_len = max([len(seq) for seq in X])

# Define paddings for the sequences
X_padding = lambda x: [[0, max_seq_len-len(x)], [0, 0], [0, 0]]
y_padding = lambda x: [[0, max_seq_len-len(x)]]

# Pad the sequences
X_padded = [tf.pad(seq, X_padding(seq)) for seq in X]
y_padded = [tf.pad(seq, y_padding(seq)) for seq in y]
w_padded = [tf.pad(seq, y_padding(seq)) for seq in w]

# Add channels dimension
X_padded = tf.expand_dims(X_padded, axis=4)

# Convert the sequences to tensors
X_tensor = tf.stack(X_padded)
y_tensor = tf.stack(y_padded)
w_tensor = tf.stack(w_padded)

# Check shapes of the sequences
print(X_tensor.shape)
print(y_tensor.shape)
print(w_tensor.shape)

(350, 4347, 10, 10, 1)
(350, 4347)
(350, 4347)


In [5]:
def build_model(num_classes, input_shape=(max_seq_len, 10, 10, 1)):

  # Input layer returning the `input_img` tensor
  input_img = layers.Input(shape=input_shape)

  # ConvLSTM2D
  x = layers.ConvLSTM2D(filters=16,
                        kernel_size=(2, 2),
                        padding='same',
                        activation='relu',
                        return_sequences=True,
                        dropout=.2)(input_img)

  # BatchNormalization
  batch_norm = layers.BatchNormalization()
  x = layers.TimeDistributed(batch_norm)(x)

  # MaxPooling2D
  max_pool = layers.MaxPool2D(pool_size=(2, 2))
  x = layers.TimeDistributed(max_pool)(x)

  # ConvLSTM2D
  x = layers.ConvLSTM2D(filters=32,
                        kernel_size=(2, 2),
                        padding='same',
                        activation='relu',
                        return_sequences=True,
                        dropout=.2)(x)

  # BatchNormalization
  batch_norm = layers.BatchNormalization()
  x = layers.TimeDistributed(batch_norm)(x)

  # MaxPooling2D
  max_pool = layers.MaxPool2D(pool_size=(2, 2))
  x = layers.TimeDistributed(max_pool)(x)

    # ConvLSTM2D
  x = layers.ConvLSTM2D(filters=64,
                        kernel_size=(2, 2),
                        padding='same',
                        activation='relu',
                        return_sequences=True,
                        dropout=.2)(x)

  # BatchNormalization
  batch_norm = layers.BatchNormalization()
  x = layers.TimeDistributed(batch_norm)(x)

  # MaxPooling2D
  max_pool = layers.MaxPool2D(pool_size=(2, 2))
  x = layers.TimeDistributed(max_pool)(x)

  # Flatten
  flatten = layers.Flatten()
  x = layers.TimeDistributed(flatten)(x)

  # Output
  dense = layers.Dense(num_classes, activation='softmax')
  output = layers.TimeDistributed(dense)(x)

  # Build the model by defining the input and output tensors
  model = Model(input_img, output)

  return model

In [6]:
model = build_model(6)
print(model.summary())

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 4347, 10, 10, 1)] 0         
_________________________________________________________________
conv_lst_m2d (ConvLSTM2D)    (None, 4347, 10, 10, 16)  4416      
_________________________________________________________________
time_distributed (TimeDistri (None, 4347, 10, 10, 16)  64        
_________________________________________________________________
time_distributed_1 (TimeDist (None, 4347, 5, 5, 16)    0         
_________________________________________________________________
conv_lst_m2d_1 (ConvLSTM2D)  (None, 4347, 5, 5, 32)    24704     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 4347, 5, 5, 32)    128       
_________________________________________________________________
time_distributed_3 (TimeDist (None, 4347, 2, 2, 32)    0     

In [7]:
learning_rate = 1e-4

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=Adam(lr=learning_rate),
              metrics=['accuracy'])

In [ ]:
history = model.fit(X_tensor,
                    y_tensor,
                    batch_size=5,
                    epochs=100,
                    validation_split=0.3,
                    sample_weight=w_tensor,
                    verbose=1)

Epoch 1/100
 3/49 [>.............................] - ETA: 1:08:41 - loss: nan - accuracy: 0.2198